# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [21]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [7]:
# number of callbacks for white-sounding name
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [6]:
data.shape

(4870, 65)

In [8]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

## 1) What test is appropriate, and does CLT apply?
We'll want to use a 2-sample t-test as we want to compare two separate groups. We also have well above 30 samples for each 'b' or 'w' and they're all in the same size (since it's categorical data), so we can make use of the central limit theorem.

## 2) What are the null and alternative hypothesis?
The null hypothesis (h0) would be that there is no discrimination between applicants based on name. This makes the alternative hypothesis (h1) that there is discrimination. 

## 3) Calculate: margin of error, confidence interval, p-value.

In [13]:
# some useful statistics for later computation:
data[data.race == 'b'].call.describe()

count    2435.000000
mean        0.064476
std         0.245649
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

In [14]:
data[data.race == 'w'].call.describe()

count    2435.000000
mean        0.096509
std         0.295346
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

In [19]:
b_std = data[data.race == 'b'].call.std()
b_mean = data[data.race == 'b'].call.mean()
w_std = data[data.race == 'w'].call.std()
w_mean = data[data.race == 'w'].call.mean()

In [4]:
b_w_ttest = stats.ttest_ind(data[data.race=='b'].call, data[data.race=='w'].call)
b_w_ttest

Ttest_indResult(statistic=-4.1147052908617514, pvalue=3.9408021031288859e-05)

In [25]:
z = 1.96 # for 95% confidence
second_term = z * (b_std / math.sqrt(data.shape[0]))
confidence_interval = (b_mean + second_term, b_mean - second_term)
confidence_interval

(0.07137572335587672, 0.057577048069370466)

In [26]:
z = 1.96 # for 95% confidence
second_term = z * (w_std / math.sqrt(data.shape[0]))
confidence_interval = (w_mean + second_term, w_mean - second_term)
confidence_interval

(0.1048043471580365, 0.08821413397733184)

margin of error at 95% confidence can be approximated in both cases using: 0.98/sqrt(sample_size). 

In [27]:
b_margin_error = 0.95/math.sqrt(data.shape[0])
b_margin_error

0.013613165508213147

In [28]:
w_margin_error = 0.95/math.sqrt(data.shape[0])
w_margin_error

0.013613165508213147

Obviously these come out the same. Is the approximation not correct or maybe doesn't apply here?

## 4) Significance

The ttest from the previous section measures wheteher an average/expe|cted value differs between the two independant samples ('b' or 'w'). We calculated a pvalue=3.9408021031288859e-05 which is very far below 0.05. This gives us enough information to reject the null hypothesis (equal call backs betwen the two groups 'b' and 'w'). 

## 5) Is race/name most important factor in callback success?

From only a ttest I don't think we have enough to say that race/name is the only factor in callback success. From the p-value acquired during the ttest we can say that it's highly unlikely that the noticeable difference is due to random chance.

Since the names/resumes are randomly assigned, does the p-value account for that randomness? Besides random assignment, it seems like it would be more thorogh to make sure that resumes with different sounding names had equivalance in the other features ('yearsexp, 'honors', 'sex', 'computerskills', etc). 

In [30]:
data.columns, len(data.columns)

(Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
        'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
        'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
        'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
        'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
        'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
        'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
        'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
        'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
        'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
        'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
        'busservice', 'othservice', 'missind', 'ownership'],
       dtype='object'), 65)

There are 65 attributes